# Ejercicio de clase regresión

1. importar las librerias necesarias

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import opendatasets as od
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

***
## 2. descargar el conjunto de datos de kaggle

In [ ]:
#dsphd = 'https://www.kaggle.com/datasets/camnugent/california-housing-prices'
#wrkds = od.download(dsphd)
#print(wrkds)

In [ ]:
#wrkdr = r'D:\codeFiles\data_science\machinel\california-housing-prices'
#os.listdir(wrkdr)

In [2]:
rawdf = pd.read_csv(r'D:\codeFiles\data_science\machinel\california-housing-prices\housing.csv')
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


***
## 3. inspección y depuración del data set

In [3]:
rawdf.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [4]:
rawdf.dropna(inplace=True)

In [5]:
rawdf.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [6]:
rawdf.ocean_proximity.unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

In [7]:
rawdf['ocean_proxomity_c'] = rawdf.ocean_proximity.map({
    'NEAR BAY': 0, '<1H OCEAN': 1, 'INLAND': 2, 'NEAR OCEAN': 3, 'ISLAND': 4
})

In [8]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
 10  ocean_proxomity_c   20433 non-null  int64  
dtypes: float64(9), int64(1), object(1)
memory usage: 1.9+ MB


In [9]:
rawdf.iloc[:,2:].groupby(by='ocean_proximity').describe()

housing_median_age                                          \
                             count       mean        std   min   25%   50%   
ocean_proximity                                                              
<1H OCEAN                   9034.0  29.277397  11.642945   2.0  20.0  30.0   
INLAND                      6496.0  24.262623  12.027144   1.0  15.0  23.0   
ISLAND                         5.0  42.400000  13.164346  27.0  29.0  52.0   
NEAR BAY                    2270.0  37.756388  13.067143   2.0  29.0  39.0   
NEAR OCEAN                  2628.0  29.314688  11.843523   2.0  20.0  29.0   

                            total_rooms               ... median_house_value  \
                  75%   max       count         mean  ...                75%   
ocean_proximity                                       ...                      
<1H OCEAN        37.0  52.0      9034.0  2627.233451  ...           289475.0   
INLAND           33.0  52.0      6496.0  2721.252925  ...           149000.0   
ISLAND           52.0  52.0         5.0  1574.600000  ...           450000.0   
NEAR BAY         52.0  52.0      2270.0  2490.335242  ...           346000.0   
NEAR OCEAN       37.0  52.0      2628.0  2587.165525  ...           322525.0   

                          ocean_proxomity_c                                     
                      max             count mean  std  min  25%  50%  75%  max  
ocean_proximity                                                                 
<1H OCEAN        500001.0            9034.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  
INLAND           500001.0            6496.0  2.0  0.0  2.0  2.0  2.0  2.0  2.0  
ISLAND           450000.0               5.0  4.0  0.0  4.0  4.0  4.0  4.0  4.0  
NEAR BAY         500001.0            2270.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
NEAR OCEAN       500001.0            2628.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0  

[5 rows x 64 columns]

In [10]:
newdf = rawdf.copy()

In [11]:
newdf.drop('ocean_proximity', axis=1, inplace=True)

***
## 4. creación del vector con las variables indicadas

In [12]:
X = newdf.iloc[:, 2:7]

In [53]:
X.head()

,housing_median_age,total_rooms,total_bedrooms,population,households
0,41.0,880.0,129.0,322.0,126.0
1,21.0,7099.0,1106.0,2401.0,1138.0
2,52.0,1467.0,190.0,496.0,177.0
3,52.0,1274.0,235.0,558.0,219.0
4,52.0,1627.0,280.0,565.0,259.0


***
<p>Crea un vector de targets con la columna 'median_house_value'</p>

pd.cut(df['points'],
                      bins=[0, 15, 25, float('Inf')],
                      labels=['Bad', 'OK', 'Good'])

In [54]:
dumpv = pd.DataFrame(newdf.median_house_value.copy())

Index(['median_house_value'], dtype='object')

In [55]:
dumpv['target'] = pd.cut(dumpv['median_house_value'], bins=[0, 206864, float('Inf')], labels=[0, 1])
dumpv.columns

In [70]:
y = pd.DataFrame(dumpv['target'].copy())
y.value_counts()

target
0         12133
1          8300
Name: count, dtype: int64

***
## 5. separar los datos en entrenamiento y prueba

In [87]:
trn_x, tst_x, trn_y, tst_y = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
#val_x, tst_x, val_y, tst_y = train_test_split(tmp_x, tmp_y, test_size=0.5, random_state=42)

***

## 6. creación del pipeline, escalar lo datos y aplicarlos a un modelo KNN 

In [97]:
mkppl = make_pipeline(StandardScaler(), KNeighborsRegressor())
mkppl.named_steps

{'standardscaler': StandardScaler(),
 'kneighborsregressor': KNeighborsRegressor()}

* Define un espacio de búsqueda de hiperparametros con 5 valores diferentes de k y con la estrategia de ponderación de distancias: uniforme y weighted.
* Haz una búsqueda de hiperparámetros Grid usando validación cruzada considerando la métrica MAE (o negative MAE). Si puedes probar también con Random Search, mejor.

In [98]:
crosv = KFold(n_splits=5, shuffle=True, random_state=42)
parms = {
    'kneighborsregressor__n_neighbors': [3,7,9,13,15],
    'kneighborsregressor__weights': ['uniform', 'distance']
}
modgd = GridSearchCV(mkppl, param_grid=parms, cv=crosv, scoring='neg_mean_squared_error')

In [99]:
modgd.fit(trn_x, trn_y)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor())]),
             param_grid={'kneighborsregressor__n_neighbors': [3, 7, 9, 13, 15],
                         'kneighborsregressor__weights': ['uniform',
                                                          'distance']},
             scoring='neg_mean_squared_error')

In [100]:
bestp = modgd.best_params_
print('mejor valor de k: ' + str(bestp['kneighborsregressor__n_neighbors']))
print(f'mejor valor de MAE: {modgd.best_score_:,.4f}')

mejor valor de k: 15
mejor valor de MAE: -0.1854


In [101]:
bestm = modgd.best_estimator_

<ul>
    <li>Con el mejor modelo, predice y obtén la métrica MAE con el conjunto de prueba.</li>
    <li>Imprime los parámetros del mejor modelo.</li>
</ul>

In [107]:
tstSx = bestm.named_steps['standardscaler'].transform(tst_x)

In [108]:
y_hat = bestm.predict(tstSx)

d:\codeFiles\data_science\machinel\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [110]:
wkRst = mean_squared_error(tst_y, y_hat)
print(wkRst)

0.26668768699725365
